In [10]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

hostname = "x-kct4.h.filess.io"
database = "olistproject_maystring"
port = "61032"
username = "olistproject_maystring"
password = "8a92da16ed9780965f7a8471dcb3232e5fb75dcd"

# Load the CSV file into a pandas DataFrame
data = pd.read_csv('/content/olist_order_payments_dataset.csv')

table_name = 'olist_order_payments'
connection = None  # Initialize connection to None

try:
    connection = mysql.connector.connect(host=hostname, database=database, user=username, password=password, port=port)
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)

        # Drop the table if it exists
        cursor.execute(f'DROP TABLE IF EXISTS {table_name};')
        print(f"Table '{table_name}' dropped if it existed.")

        # Create the table
        create_table_query = f"""
        CREATE TABLE {table_name} (
          order_id VARCHAR(50),
          payment_sequential INT,
          payment_type VARCHAR(50), # Added payment_type column as per the INSERT query
          payment_installments INT,
          payment_value FLOAT
        );
        """

        cursor.execute(create_table_query)
        print(f"Table '{table_name}' created successfully!")

        batch_size = 1000
        total_records = len(data)

        print(f"Starting data insertion into '{table_name}' in batches of {batch_size} records")

        for start in range(0, total_records, batch_size):
          end = start + batch_size
          batch = data.iloc[start:end] # Get the current batch of records

          batch_records = [
            tuple(row) for row in batch.itertuples(index = False, name = None)
          ]

          insert_query = f"""
          INSERT INTO {table_name}
          (order_id, payment_sequential, payment_type, payment_installments, payment_value)
          VALUES (%s, %s, %s, %s, %s);
          """

          cursor.executemany(insert_query, batch_records)
          connection.commit()
          print(f"inserted records {start + 1} to {min(end, total_records)} successfully")

        print(f"All {total_records} records inserted successfully into '{table_name}' ")

except Error as e:
    print("Error while connecting to MySQL", e)
except Exception as e: # Catch any other potential errors during execution
    print("An error occurred:", e)

finally:
    if connection is not None and connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

/tmp/ipython-input-817211826.py:20: DeprecationWarning: Call to deprecated function get_server_info. Reason: 
    The property counterpart 'server_info' should be used instead.

  db_Info = connection.get_server_info()


Connected to MySQL Server version  8.0.36-28
You're connected to database:  ('olistproject_maystring',)
Table 'olist_order_payments' dropped if it existed.
Table 'olist_order_payments' created successfully!
Starting data insertion into 'olist_order_payments' in batches of 1000 records
inserted records 1 to 1000 successfully
inserted records 1001 to 2000 successfully
inserted records 2001 to 3000 successfully
inserted records 3001 to 4000 successfully
inserted records 4001 to 5000 successfully
inserted records 5001 to 6000 successfully
inserted records 6001 to 7000 successfully
inserted records 7001 to 8000 successfully
inserted records 8001 to 9000 successfully
inserted records 9001 to 10000 successfully
inserted records 10001 to 11000 successfully
inserted records 11001 to 12000 successfully
inserted records 12001 to 13000 successfully
inserted records 13001 to 14000 successfully
inserted records 14001 to 15000 successfully
inserted records 15001 to 16000 successfully
inserted records

In [8]:
orders_payments = pd.read_csv('/content/olist_order_payments_dataset.csv')
orders_payments.head()
orders_payments.shape

(103886, 5)